# import

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import math
import glob
import re
import random
import collections
import os
import sys
import pickle

# data

In [2]:
botname = 'แว่นเป๋อ❄️'
in_folder = '/data2/backup/'
out_folder = '/'.join(in_folder.split('/')[:-2])+'/'

In [12]:
import os
convs = []
for file in os.listdir(in_folder):
    if(file in ["out.txt"]):
        continue
    with open(in_folder+file) as f:
        print(file)

        conv = []
        isFirst = True
        isMultipleLine = False
        is_human_first = True

        combined_sent = []
        prev_send = ''
        
        for line in f:            
            sender = ''
            sent = ''
            spl = line.strip().split('\t')
            
            if(len(spl) == 3):
                sender = spl[1]
                sent = spl[2]
                
                if(isFirst):
                    if(sender == botname):
                        is_human_first = False
                    isFirst = False

                if(sent in ['[Photo]','[Sticker]','[File]' , '[Video]']):
                    continue
                if(sent[0] == '☎'):
                    continue
                
                if(sent[0] == '"'):
                    sent = sent[1:]
                    isMultipleLine = True
                    
                if(sender != prev_send):
                    conv.append(prev_send + "\t" +' '.join(combined_sent))
                    combined_sent = [sent]
                    prev_send = sender
                    
                else:
                    combined_sent.append(sent)
            
            elif(isMultipleLine):
                if(len(line) > 1 and line[-2] == '"'):
                    isMultipleLine = False
                combined_sent.append(line.strip().strip('"'))
                

#         combined_sent.append(sent)
        if(prev_send != botname):
            conv.append(prev_send + '\t' +' '.join(combined_sent))
        
        
        if( is_human_first ):
            conv = conv[1:]
        else:
            conv = conv[2:]
        
        convs.append(conv[:-1])
        
with open(out_folder+'debug-sentences.txt', 'w') as o:
    for con in convs:
        for sent in con:
            o.write(sent + '\n')
            
with open(out_folder+'sentences.txt', 'w') as o:
    for con in convs:
        for sent in con:
            o.write(sent.split('\t')[1] + '\n')            

[LINE] Chat with mos.txt
[LINE] Chat with Fair.txt
[LINE] Chat with KNT.txt
[LINE] Chat with NoTT'z 🔩.txt
LINE Chat with Jaja 2559 (1).txt
[LINE] Chat with OaT.txt
[LINE] Chat with Ten'ten.txt


In [ ]:
out_path = '/data2/'

In [ ]:
conver = []
with open(out_path+'sentences.txt') as f:
    for line in f:
        conver.append(line)

# tokenize

In [ ]:
import deepcut
deepcut.tokenize("สวัสดีเพื่อนๆ")

In [ ]:
%%time
token = []
for sent in conver:
    token.append(deepcut.tokenize(sent.strip()))
print(token[:5])

In [ ]:
token_set = set()
for words in token:
    token_set = token_set.union(words)

token_list = list(token_set)
print(token_list)

In [ ]:
def create_dataset_dict(input_list):
    words = []
    for review in input_list:
        for word in review:
            words.append(word)
    word_count = list()
    #use set and len to get the number of unique words
    word_count.extend(collections.Counter(words).most_common(len(set(words))))
    
    #threshold = len(input_text)//1e6
    threshold = 3
    print("Threshold", threshold)
    
    word_unk = set()
    num_unk = 0
    idx = len(word_count)-1
    while word_count[idx][1] <= threshold:
        word_unk.add(word_count[idx][0])
        num_unk += word_count[idx][1]
        idx -= 1

    word_count = word_count[:idx+1]
    
    #include a token for unknown word
    word_count.append(("UNK",num_unk))
    word_count = sorted(word_count, key=lambda x: -x[1])
    #word_count = [x for x in word_count if x[0] not in word_unk]
    
    #print out 10 most frequent words
    print(word_count[:10])
    print("#unk" ,num_unk)
    
    dictionary = dict()
    dictionary["for_keras_zero_padding"] = 0
    for word in word_count:
        dictionary[word[0]] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    datas = list()
    for review in input_list:
        data = []
        for word in review:
            if word in word_unk:
                data.append(dictionary["UNK"])
            else:
                data.append(dictionary[word])
        datas.append(data)

    return datas,dictionary, reverse_dictionary

In [ ]:
dataset,dictionary, reverse_dictionary=create_dataset_dict(token)

In [ ]:
print("len data set:", len(dataset))
print("len dictionary:", len(dictionary))
print("output sample (dataset):",dataset[:10])
print("output sample (dictionary):",{k: dictionary[k] for k in list(dictionary)[:10]}, "\ndictionary size: ",len(dictionary))
print("output sample (reverse dictionary):",{k: reverse_dictionary[k] for k in list(reverse_dictionary)[:10]})

In [ ]:
human = []
bot = []
even = True
for line in dataset:
    if (even):
        human.append(line)
        even = False
    else:
        bot.append(line)
        even = True
test_train_split = 0.8
        
human_train = human[:int(len(human)*test_train_split)]
human_test = human[int(len(human)*test_train_split):]
bot_train = bot[:int(len(bot)*test_train_split)]
bot_test = bot[int(len(bot)*test_train_split):]

with open(out_path+'human_train.txt', 'w') as o:
    for line in human_train:
        line = " ".join([str(x) for x in line])
        o.write(line+'\n')
with open(out_path+'bot_train.txt', 'w') as o:
    for line in bot_train:
        line = " ".join([str(x) for x in line])
        o.write(line+'\n')
with open(out_path+'human_test.txt', 'w') as o:
    for line in human_test:
        line = " ".join([str(x) for x in line])
        o.write(line+'\n')
with open(out_path+'bot_test.txt', 'w') as o:
    for line in bot_test:
        line = " ".join([str(x) for x in line])
        o.write(line+'\n')

In [ ]:
dict_list = []
with open(out_path+'vocab.txt','w') as o:
    for value in dictionary:
        o.write(value+'\n')